## Programming Assignment: Рекомендательные системы

### Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать **recall@k** и **precision@k**.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

### Входные данные

Вам дается две выборки с пользовательскими сессиями - **id**-шниками просмотренных и **id**-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: **id** просмотренных товаров через **,** затем идёт **;** после чего следуют **id** купленных товаров (если такие имеются), разделённые запятой. Например, **1,2,3,4;** или **1,2,3,4;5,6**.

Гарантируется, что среди **id** купленных товаров все различные.

### Важно:

- Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.

- Если товар не встречался в обучающей выборке, его популярность равна 0.
 
- Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.

- Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и **k** в **recall@k / precision@k**.

### Задание

1.На обучении постройте частоты появления **id** в просмотренных и в купленных (**id** может несколько раз появляться в просмотренных, все появления надо учитывать)

2.Реализуйте два алгоритма рекомендаций:

- сортировка просмотренных **id** по популярности (частота появления в просмотренных),

- сортировка просмотренных **id** по покупаемости (частота появления в покупках).

3.Для данных алгоритмов выпишите через пробел **AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5** на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

### Дополнительные вопросы

1.Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров **recall@k** меняется. Подумайте, как оценить минимальное и максимальное значение **recall@k** в зависимости от правила сортировки.

2.Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [2]:
file_sessions_train = 'coursera_sessions_train.txt'
file_sessions_test = 'coursera_sessions_test.txt'

In [26]:
COLUMN_NAME_VIEW = 'freq_view'
COLUMN_NAME_PURCHASE = 'freq_purchase'

In [33]:
train_data = pd.read_csv(file_sessions_train, sep=';', names=['view', 'purchase'])
test_data = pd.read_csv(file_sessions_test, sep=';', names=['view', 'purchase'])
train_data.head(3)

,view,purchase
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN


In [63]:
def read_sessions(data):
    data_freq = pd.DataFrame(columns=[COLUMN_NAME_VIEW, COLUMN_NAME_PURCHASE])
    
    for row in data.iterrows():
        view_items = row[1]['view'].split(',')
        purchase_items = []
        if pd.isnull(row[1]['purchase']) == False:
            purchase_items = row[1]['purchase'].split(',')
        
        for item in view_items:
            if item in data_freq.index:
                data_freq.loc[item, COLUMN_NAME_VIEW] += 1
            else:
                data_freq = data_freq.append(pd.Series({COLUMN_NAME_VIEW:0, COLUMN_NAME_PURCHASE:0}, name=item))
                
        for item in purchase_items:
            if item in data_freq.index:
                data_freq.loc[item, COLUMN_NAME_PURCHASE] += 1
            else:
                data_freq = data_freq.append(pd.Series({COLUMN_NAME_VIEW:0, COLUMN_NAME_PURCHASE:0}, name=item))  
                
    return data_freq

In [64]:
%%time
freq_train = read_sessions(train_data)

CPU times: user 11min 48s, sys: 1 s, total: 11min 49s
Wall time: 11min 49s


In [172]:
# Возвращает список id без повторений
def get_ids(ids):
    new_ids = []
    for id_ in ids:
        if id_ not in new_ids:
            new_ids.append(id_)
    return new_ids

In [193]:
# Возвращает отсортированный список ids.
# Сортировка по просмотренным товарам (sort_by = 'freq_view') 
# Сортировка по купленным товарам (sort_by='freq_purchase)
# Сортируется по таблице data_freq.
def get_recommendations(ids, data_freq, sort_by, k):
    ids = get_ids(ids)
    recomendations_count = min(k, len(ids))
    # таблица со значениями id и значениями ранга
    freq_ids = pd.DataFrame(columns=['order', COLUMN_NAME_VIEW, COLUMN_NAME_PURCHASE])
    
    order = 0
    row = pd.Series()
    
    for id_ in ids:
        if id_ in data_freq.index:
            row = data_freq.loc[id_, :].append(pd.Series({'order': order}))
        else:
            row = pd.Series({COLUMN_NAME_VIEW: 0, COLUMN_NAME_PURCHASE: 0, 'order': order})
        
        row.name = id_
        freq_ids = freq_ids.append(row)
        order += 1

    freq_ids.sort_values(by=[sort_by, 'order'], ascending=[False, True], inplace=True)
    return freq_ids.index[:recomendations_count]

In [194]:
def get_metrics(data, k, data_freq, sort_by):
    recall = np.array([])
    precision = np.array([])
    
    data = data.dropna()
    for row in data.iterrows():
        view_items = row[1]['view'].split(',')
        purchase_items = row[1]['purchase'].split(',')
        
        recomendations = get_recommendations(view_items, data_freq, sort_by, k)
        purchase_from_recomendations = set.intersection(set(recomendations), set(purchase_items))

        precision = np.append(precision, float(len(purchase_from_recomendations))/k)
        recall = np.append(recall, float(len(purchase_from_recomendations))/len(purchase_items)) 
        
    return (np.mean(recall), np.mean(precision))

In [252]:
# Итоговая таблица результатов
columns = ['recommendations', 'sample', 'by_view_popularity', 'by_purchase_popularity']
result_recall = pd.DataFrame(columns=columns)
result_precision = pd.DataFrame(columns=columns)

**Сортировка по частототе появления в просмотренных (обучающая выборка):**

In [253]:
%%time
recall_1, precision_1 = get_metrics(data=train_data, k=1, data_freq=freq_train, sort_by=COLUMN_NAME_VIEW)
recall_5, precision_5 = get_metrics(data=train_data, k=5, data_freq=freq_train, sort_by=COLUMN_NAME_VIEW)

CPU times: user 1min 56s, sys: 80 ms, total: 1min 57s
Wall time: 1min 57s


In [254]:
result_recall = result_recall.append({'recommendations':1, 'by_view_popularity':recall_1, 
                                      'sample': 'train'}, ignore_index=True)
result_precision = result_precision.append({'recommendations':1, 'by_view_popularity':precision_1, 
                                            'sample': 'train'}, ignore_index=True)

In [255]:
result_recall = result_recall.append({'recommendations':5, 'by_view_popularity':recall_5, 
                                      'sample': 'train'}, ignore_index=True)
result_precision = result_precision.append({'recommendations':5, 'by_view_popularity':precision_5, 
                                            'sample': 'train'}, ignore_index=True)

In [256]:
answers = [recall_1, precision_1, recall_5, precision_5]
with open("views_popularity_train.txt", "w") as fout:
    fout.write(" ".join(["%.2f"%num for num in answers]))

**Сортировка по частоте появления в покупках (обучающая выборка):**

In [258]:
%%time
recall_1, precision_1 = get_metrics(data=train_data, k=1, data_freq=freq_train, sort_by=COLUMN_NAME_PURCHASE)
recall_5, precision_5 = get_metrics(data=train_data, k=5, data_freq=freq_train, sort_by=COLUMN_NAME_PURCHASE)

CPU times: user 1min 55s, sys: 116 ms, total: 1min 55s
Wall time: 1min 55s


In [272]:
index = result_recall[(result_recall['recommendations']==1)&(result_recall['sample']=='train')].index
result_recall.loc[index, 'by_purchase_popularity'] = recall_1

index = result_recall[(result_recall['recommendations']==5)&(result_recall['sample']=='train')].index
result_recall.loc[index, 'by_purchase_popularity'] = recall_5

index = result_precision[(result_precision['recommendations']==1)&(result_precision['sample']=='train')].index
result_precision.loc[index, 'by_purchase_popularity'] = precision_1

index = result_precision[(result_precision['recommendations']==5)&(result_precision['sample']=='train')].index
result_precision.loc[index, 'by_purchase_popularity'] = precision_5

In [275]:
answers = [recall_1, precision_1, recall_5, precision_5]
with open("purchases_popularity_train.txt", "w") as fout:
    fout.write(" ".join(["%.2f"%num for num in answers]))

**Сортировка по частототе появления в просмотренных (тестовая выборка):**

In [276]:
%%time
recall_1, precision_1 = get_metrics(data=test_data, k=1, data_freq=freq_train, sort_by=COLUMN_NAME_VIEW)
recall_5, precision_5 = get_metrics(data=test_data, k=5, data_freq=freq_train, sort_by=COLUMN_NAME_VIEW)

CPU times: user 2min 6s, sys: 128 ms, total: 2min 6s
Wall time: 2min 6s


In [277]:
result_recall = result_recall.append({'recommendations':1, 'by_view_popularity':recall_1, 
                                      'sample': 'test'}, ignore_index=True)
result_precision = result_precision.append({'recommendations':1, 'by_view_popularity':precision_1, 
                                            'sample': 'test'}, ignore_index=True)

In [278]:
result_recall = result_recall.append({'recommendations':5, 'by_view_popularity':recall_5, 
                                      'sample': 'test'}, ignore_index=True)
result_precision = result_precision.append({'recommendations':5, 'by_view_popularity':precision_5, 
                                            'sample': 'test'}, ignore_index=True)

In [279]:
answers = [recall_1, precision_1, recall_5, precision_5]
with open("views_popularity_test.txt", "w") as fout:
    fout.write(" ".join(["%.2f"%num for num in answers]))

**Сортировка по частоте появления в покупках (тестовая выборка):**

In [283]:
%%time
recall_1, precision_1 = get_metrics(data=test_data, k=1, data_freq=freq_train, sort_by=COLUMN_NAME_PURCHASE)
recall_5, precision_5 = get_metrics(data=test_data, k=5, data_freq=freq_train, sort_by=COLUMN_NAME_PURCHASE)

CPU times: user 2min 11s, sys: 136 ms, total: 2min 11s
Wall time: 2min 12s


In [284]:
index = result_recall[(result_recall['recommendations']==1)&(result_recall['sample']=='test')].index
result_recall.loc[index, 'by_purchase_popularity'] = recall_1

index = result_recall[(result_recall['recommendations']==5)&(result_recall['sample']=='test')].index
result_recall.loc[index, 'by_purchase_popularity'] = recall_5

index = result_precision[(result_precision['recommendations']==1)&(result_precision['sample']=='test')].index
result_precision.loc[index, 'by_purchase_popularity'] = precision_1

index = result_precision[(result_precision['recommendations']==5)&(result_precision['sample']=='test')].index
result_precision.loc[index, 'by_purchase_popularity'] = precision_5

In [285]:
answers = [recall_1, precision_1, recall_5, precision_5]
with open("purchases_popularity_test.txt", "w") as fout:
    fout.write(" ".join(["%.2f"%num for num in answers]))

In [286]:
result_recall

,recommendations,sample,by_view_popularity,by_purchase_popularity
0,1.0,train,0.442634,0.689558
1,5.0,train,0.824692,0.926501
2,1.0,test,0.418447,0.460152
3,5.0,test,0.800734,0.820142


In [287]:
result_precision

,recommendations,sample,by_view_popularity,by_purchase_popularity
0,1.0,train,0.512195,0.804878
1,5.0,train,0.212528,0.252716
2,1.0,test,0.482401,0.527422
3,5.0,test,0.203984,0.210095
